<a href="https://colab.research.google.com/github/Tatsuro0726/chemoinfomatics/blob/main/deepchem/Tutorial10_create_dataset_from_experimental_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Tutorial:10
実験データからのデータセットの作成方法

#### Setup

In [1]:
!curl -Lo conda_installer.py https://raw.githubusercontent.com/deepchem/deepchem/master/scripts/colab_install.py
import conda_installer
conda_installer.install()
!/root/miniconda/bin/conda info -e

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3490  100  3490    0     0  13632      0 --:--:-- --:--:-- --:--:-- 13632


add /root/miniconda/lib/python3.6/site-packages to PYTHONPATH
python version: 3.6.9
fetching installer from https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
done
installing miniconda to /root/miniconda
done
installing rdkit, openmm, pdbfixer
added omnia to channels
added conda-forge to channels
done
conda packages installation finished!


# conda environments:
#
base                  *  /root/miniconda



In [2]:
!pip install --pre deepchem
import deepchem
deepchem.__version__

     |████████████████████████████████| 532kB 13.3MB/s 


'2.5.0.dev'

#### データファイルでの作業

実験データから機械学習モデルを構築する場合を考える。

 - 構築する際の課題
    - データの分解・解析
    - データのノイズ除去

- 入力フォーマット  
    - DeepChemでは、.csv, .sdf, .fasta, .png,などをサポート。

- Featurization
    - 分子をベクトルに変換するためにFeaturizerオブジェクトを渡す必要がある。

- データの解析
    - データ読み込みのため、Pandasを使用
    - 薬物名をSMILESに変換するためpubchempyを使用。



In [5]:
# pubchempyを使用
!/root/miniconda/bin/conda install pubchempy

Solving environment: | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ done

# All requested packages already installed.



In [8]:
import os
import pandas as pd
from pubchempy import get_cids, get_compounds

In [6]:
import os
from IPython.display import Image, display
current_dir = os.path.dirname(os.path.realpath('__file__'))

In [7]:
# xlsxファイルのダウンロード
import deepchem as dc
dc.utils.download_url(
    'https://github.com/deepchem/deepchem/raw/master/datasets/Positive%20Modulators%20Summary_%20918.TUC%20_%20v1.xlsx',
    current_dir,
    'Positive Modulators Summary_ 918.TUC _ v1.xlsx'
)

In [9]:
# rawデータファイルの読み込み
raw_data_file = os.path.join(current_dir, 'Positive Modulators Summary_ 918.TUC _ v1.xlsx')
raw_data_excel = pd.ExcelFile(raw_data_file)

# second sheet only
raw_data = raw_data_excel.parse(raw_data_excel.sheet_names[1])

In [10]:
# 5行確認
raw_data.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Metric #1 (-120 mV Peak),Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
0,NaN,NaN,NaN,Vehicle,NaN,4,Replications,NaN
1,TA ##,Position,TA ID,Mean,SD,Threshold (%) = Mean + 4xSD,N #1 (%),N #2 (%)
2,1,1-A02,Penicillin V Potassium,-12.8689,6.74705,14.1193,-10.404,-18.1929
3,2,1-A03,Mycophenolate Mofetil,-12.8689,6.74705,14.1193,-12.4453,-11.7175
4,3,1-A04,Metaxalone,-12.8689,6.74705,14.1193,-8.65572,-17.7753


In [11]:
# 不要なカラムを削除
raw_data = raw_data.iloc[2:, [2,6,7]]

# インデックスのリセット
raw_data.reset_index(inplace=True)

# カラムのリネーム
raw_data.columns = ['label', 'drug', 'n1','n2']

In [12]:
raw_data.head(5)

,label,drug,n1,n2
0,2,Penicillin V Potassium,-10.404,-18.1929
1,3,Mycophenolate Mofetil,-12.4453,-11.7175
2,4,Metaxalone,-8.65572,-17.7753
3,5,Terazosin·HCl,-11.5048,16.0825
4,6,Fluvastatin·Na,-11.1354,-14.553


In [13]:
# 薬物名を抽出
drugs = raw_data['drug'].values
print(drugs[:5])

['Penicillin V Potassium' 'Mycophenolate Mofetil' 'Metaxalone'
 'Terazosin·HCl' 'Fluvastatin·Na']


In [14]:
# Compoundオブジェクトを取得
get_compounds(drugs[1], 'name')

[Compound(5281078)]

In [16]:
# smiles取得
get_compounds(drugs[1], 'name')[0].canonical_smiles

'CC1=C2COC(=O)C2=C(C(=C1OC)CC=C(C)CCC(=O)OCCN3CCOCC3)O'

In [20]:
import re

ion_replacements = {
    'HBr': ' hydrobromide',
    '2Br': ' dibromide',
    'Br': ' bromide',
    'HCl': ' hydrochloride',
    '2H2O': ' dihydrate',
    'H20': ' hydrate',
    'Na': ' sodium'
}

ion_keys = ['H20', 'HBr', 'HCl', '2Br', '2H2O', 'Br', 'Na']

def compound_to_smiles(cmpd):
    # remove spaces and irregular characters
    compound = re.sub(r'([^\s\w]|_)+', '', cmpd)
                   
    # replace ion names if needed
    for ion in ion_keys:
        if ion in compound:
            compound = compound.replace(ion, ion_replacements[ion])

    # query for cid first in order to avoid timeouterror
    cid = get_cids(compound, 'name')[0]
    smiles = get_compounds(cid)[0].canonical_smiles

    return smiles

In [23]:
# smiles取得
# エラーが発生する場合もあるためtry-except
smiles_map = {}
for i, compound in enumerate(drugs):
    try:
        smiles_map[compound] = compound_to_smiles(compound)
    except:
        print('Error:%s' % i)
        continue

In [24]:
smiles_data = raw_data
# map drug to smiles string
smiles_data['drug'] = smiles_data['drug'].apply(lambda x: smiles_map[x])

In [25]:
# 先頭5行確認
smiles_data.head(5)

,label,drug,n1,n2
0,2,CC1(C(N2C(S1)C(C2=O)NC(=O)COC3=CC=CC=C3)C(=O)[...,-10.404,-18.1929
1,3,CC1=C2COC(=O)C2=C(C(=C1OC)CC=C(C)CCC(=O)OCCN3C...,-12.4453,-11.7175
2,4,CC1=CC(=CC(=C1)OCC2CNC(=O)O2)C,-8.65572,-17.7753
3,5,COC1=C(C=C2C(=C1)C(=NC(=N2)N3CCN(CC3)C(=O)C4CC...,-11.5048,16.0825
4,6,CC(C)N1C2=CC=CC=C2C(=C1C=CC(CC(CC(=O)[O-])O)O)...,-11.1354,-14.553


#### ノイズ除去
- 機械学習における万能な方法はない(no free lunch)
- 解きたい問題に対する適切なモデルのフレームワークを決めるために、データの分析と理解に時間を費やす必要がある。
    - 何を実現したいのか？
    - アッセイは何ですか？
    - データ構造はどのような形式？
    - データに意味はありますか？
    - 従来何が取り組まれていますか？

- このプロジェクトでは。。
    - 任意の低分子について、特定のイオンチャネルタンパク質との親和性を予測できるモデルを構築したい
    - 入力の薬物に対して、データはチャネル阻害を意味する
    - 数百の薬物データ
    - データセットについて詳しくみる必要がある
    - 特定のタンパク質については何もない
    

In [26]:
# データを詳細に眺める
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set_style('white')

import numpy as np

from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw, PyMol, rdFMCS
from rdkit.Chem.Draw import IPythonConsole
from rdkit import rdBase
